<font size="8"> **Adding environmental data from available observations to unique crabeater observations** </font>  
In this notebook, we will extract environmental data from sea ice and sea surface temperature observations using the coordinates of the reported crabeater seals sightings.

# Setting working directory
In order to ensure these notebooks work correctly, we will set the working directory. We assume that you have saved a copy of this repository in your home directory (represented by `~` in the code chunk below). If you have saved this repository elsewhere in your machine, you need to ensure you update this line with the correct filepath where you saved these notebooks.

In [1]:
import os
os.chdir(os.path.expanduser('~/Chapter2_Crabeaters/Scripts'))

# Loading other relevant libraries

In [2]:
from dask.distributed import Client
from glob import glob
#Accessing model data
import cosima_cookbook as cc
#Useful functions
import UsefulFunctions as uf
#Dealing with data
import xarray as xr
import pandas as pd
import numpy as np
#Data visualisation
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

# Paralellising work 

In [148]:
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 7
Total threads: 14,Total memory: 63.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:42771,Workers: 7
Dashboard: /proxy/8787/status,Total threads: 14
Started: Just now,Total memory: 63.00 GiB
Comm: tcp://127.0.0.1:40835,Total threads: 2
Dashboard: /proxy/43123/status,Memory: 9.00 GiB
Nanny: tcp://127.0.0.1:36603,


# Loading unique crabeater seal observations data frame

In [4]:
#Loading dataset as pandas data frame
crabeaters = pd.read_csv('../Biological_Data/unique_crabeater_obs_grid.csv')

#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)

#Checking results
crabeaters

,event_date,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,decade,sector,zone,presence
0,2019-03-09T00:00:00Z,2019-03,2019,3,-64.888,-62.85,-64.910,-62.9,autumn,in-between,2010,East Pacific,Antarctic,1
1,2019-03-13T00:00:00Z,2019-03,2019,3,-65.142,-64.15,-65.121,-64.1,autumn,in-between,2010,East Pacific,Antarctic,1
2,2019-03-15T00:00:00Z,2019-03,2019,3,-65.142,-64.05,-65.121,-64.1,autumn,in-between,2010,East Pacific,Antarctic,1
3,2019-03-15T00:00:00Z,2019-03,2019,3,-65.100,-63.95,-65.079,-64.0,autumn,in-between,2010,East Pacific,Antarctic,1
4,2019-03-14T00:00:00Z,2019-03,2019,3,-65.353,-64.15,-65.332,-64.1,autumn,in-between,2010,East Pacific,Antarctic,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3975,2006-02-19T13:00:00Z,2006-02,2006,2,-67.085,69.95,-67.064,70.0,summer,moulting,2000,Central Indian,Antarctic,1
3976,2006-01-30T13:00:00Z,2006-01,2006,1,-67.972,40.75,-67.951,40.7,summer,moulting,2000,Central Indian,Antarctic,1
3977,2006-02-19T13:00:00Z,2006-02,2006,2,-67.085,69.95,-67.106,70.0,summer,moulting,2000,Central Indian,Antarctic,1
3978,2006-02-19T13:00:00Z,2006-02,2006,2,-67.127,69.95,-67.149,70.0,summer,moulting,2000,Central Indian,Antarctic,1


# Adding values for static variables only
Static variables referred to any physical variables that do not change over time (at least not during the time period of our interest). Examples include depth of the water column and distance to coastline. Given that we only have one value for these variables, the process of extracting data is relatively simple. We do not need to take into account the date observations were collected.

## Defining dictionary with information about static variables
This dictionary contains the column labels for each and the name of the files for each static variable to be included in our analysis. We will also define a variable containing the full path to the folder where all static variables are stored.

In [6]:
#Full path to static variables
base_dir_static = '/g/data/v45/la6889/Chapter2_Crabeaters/Static_Variables/'

#List of static variables
varDict = {'bottom_slope_deg': 'bathy_slope_GEBCO_2D.nc',
           'dist_shelf_km': 'distance_shelf.nc',
           'dist_coast_km': 'distance_coastline.nc',
           'depth_m': 'bathy_GEBCO_2D.nc'}

## Extracting data for each observation and adding it to a new column in crabeater data

In [7]:
#Getting coordinates from crabeater dataset
lat = xr.DataArray(crabeaters.latitude.values)
lon = xr.DataArray(crabeaters.longitude.values)

#Looping through dictionary keys
for var in varDict:
    #Creating full path to file of interest
    file_path = os.path.join(base_dir_static, varDict[var])
    #Load as raster
    ras = xr.open_dataarray(file_path)
    rename_var = ras.name
    #Extracting values
    ras_sub = ras.sel(xt_ocean = lon, yt_ocean = lat, method = 'nearest')
    #Turning into data frame and rounding all columns to 3 decimal places
    ras_df = ras_sub.to_dataframe().round(3).rename(columns = {rename_var: var})
    #Adding to crabeater observations data frame
    crabeaters = pd.merge(crabeaters, ras_df, on = ['yt_ocean', 'xt_ocean'], how = 'left', sort = True).drop_duplicates()
    
#Checking results
crabeaters

,event_date,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,decade,sector,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,2004-02-19T00:00:00Z,2004-02,2004,2,-77.687,166.35,-77.708,166.3,summer,moulting,2000,West Pacific,Antarctic,1,89.980469,606.013921,32.155391,176.033340
1,2014-01-13T14:51:22Z,2014-01,2014,1,-76.842,-36.05,-76.864,-36.1,summer,moulting,2010,Atlantic,Antarctic,1,89.729942,383.202891,41.998688,1064.320435
2,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-157.25,-76.779,-157.3,summer,moulting,2000,West Pacific,Antarctic,1,89.943176,113.770361,4.724272,612.181458
3,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-156.85,-76.779,-156.8,summer,moulting,2000,West Pacific,Antarctic,1,89.897667,113.278688,2.641858,656.099976
4,2020-02-02T00:00:00Z,2020-02,2020,2,-76.378,163.95,-76.357,163.9,summer,moulting,2020,West Pacific,Antarctic,1,89.871246,603.310632,21.026712,519.266663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3975,1989-11-05T10:00:00Z,1989-11,1989,11,-60.047,70.25,-60.072,70.2,autumn,weaning,1980,Central Indian,Subantarctic,1,89.978432,478.561536,622.374268,4235.086426
3976,2008-12-26T00:00:00Z,2008-12,2008,12,-59.645,9.15,-59.620,9.1,summer,weaning,2000,Atlantic,Antarctic,1,89.958000,844.112672,926.684712,5462.944336
3977,1989-11-05T02:00:00Z,1989-11,1989,11,-59.493,71.85,-59.468,71.9,autumn,weaning,1980,Central Indian,Subantarctic,1,89.864761,531.076147,692.923480,4498.345215
3978,2013-11-12T00:00:00Z,2013-11,2013,11,-57.829,-26.35,-57.803,-26.3,autumn,weaning,2010,Atlantic,Antarctic,1,NaN,1443.071239,1067.076760,506.524384


## Saving data frame with static variables
Given that the dynamic variables take some time to extract. We will save intermediary results to avoid having to extract them again.

In [8]:
#Defining output folder
folder_out = '../Environmental_Data/Env_obs'
#Checking folder exists
os.makedirs(folder_out, exist_ok = True)

#Saving file
crabeaters.to_csv(os.path.join(folder_out, 'unique_crabeater_obs_static_env.csv'), index = False)

# Adding values for dynamic variables
Given the amount of crabeater seal observations and the time period covered by this dataset, the extraction of these values may take some time. It is recommended to save the data frame after every time a new variable is extracted. This way we can avoid losing data.

In [9]:
crabeaters = pd.read_csv(os.path.join(folder_out, 'unique_crabeater_obs_static_env.csv'))
#Ensuring date column is formatted correctly (year-month)
crabeaters['date'] = crabeaters.apply(lambda x: f'{x.year}-{str(x.month).zfill(2)}', axis = 1)
crabeaters

,event_date,date,year,month,yt_ocean,xt_ocean,yu_ocean,xu_ocean,season_year,life_stage,decade,sector,zone,presence,bottom_slope_deg,dist_shelf_km,dist_coast_km,depth_m
0,2004-02-19T00:00:00Z,2004-02,2004,2,-77.687,166.35,-77.708,166.3,summer,moulting,2000,West Pacific,Antarctic,1,89.980469,606.013921,32.155391,176.033340
1,2014-01-13T14:51:22Z,2014-01,2014,1,-76.842,-36.05,-76.864,-36.1,summer,moulting,2010,Atlantic,Antarctic,1,89.729942,383.202891,41.998688,1064.320435
2,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-157.25,-76.779,-157.3,summer,moulting,2000,West Pacific,Antarctic,1,89.943176,113.770361,4.724272,612.181458
3,2000-01-06T00:00:00Z,2000-01,2000,1,-76.758,-156.85,-76.779,-156.8,summer,moulting,2000,West Pacific,Antarctic,1,89.897667,113.278688,2.641858,656.099976
4,2020-02-02T00:00:00Z,2020-02,2020,2,-76.378,163.95,-76.357,163.9,summer,moulting,2020,West Pacific,Antarctic,1,89.871246,603.310632,21.026712,519.266663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3975,1989-11-05T10:00:00Z,1989-11,1989,11,-60.047,70.25,-60.072,70.2,autumn,weaning,1980,Central Indian,Subantarctic,1,89.978432,478.561536,622.374268,4235.086426
3976,2008-12-26T00:00:00Z,2008-12,2008,12,-59.645,9.15,-59.620,9.1,summer,weaning,2000,Atlantic,Antarctic,1,89.958000,844.112672,926.684712,5462.944336
3977,1989-11-05T02:00:00Z,1989-11,1989,11,-59.493,71.85,-59.468,71.9,autumn,weaning,1980,Central Indian,Subantarctic,1,89.864761,531.076147,692.923480,4498.345215
3978,2013-11-12T00:00:00Z,2013-11,2013,11,-57.829,-26.35,-57.803,-26.3,autumn,weaning,2010,Atlantic,Antarctic,1,NaN,1443.071239,1067.076760,506.524384


## Loading environmental data from observations

In [138]:
#Creating dictionary with useful information
varDict = {'var_name': 'dist_ice_edge_km',
           #Folder containing obs
           'obs_main': '/g/data/v45/la6889/Chapter2_Crabeaters/SeaIceObs/Distance_Edge/*.nc',
           #Output folder
           'base_out': '../Environmental_Data/Env_obs'}

In [141]:
#Getting list of all obs in folder
files_var = sorted(glob(varDict['obs_main']))

#Loading all data into single dataset
var_df = xr.open_mfdataset(files_var)
var_df = var_df.rename_vars({'dist_km': 'dist_ice_edge_km'})
var_df = var_df.dist_ice_edge_km.rename({'lon': 'xt_ocean', 'lat': 'yt_ocean'})

#Checking results
var_df

<xarray.DataArray 'dist_ice_edge_km' (time: 494, yt_ocean: 740, xt_ocean: 3600)>
dask.array<concatenate, shape=(494, 740, 3600), dtype=float64, chunksize=(1, 740, 3600), chunktype=numpy.ndarray>
Coordinates:
  * time      (time) datetime64[ns] 1978-11-15 1978-12-15 ... 2019-12-15
  * yt_ocean  (yt_ocean) float64 -81.11 -81.07 -81.02 ... -45.18 -45.11 -45.04
  * xt_ocean  (xt_ocean) float64 -179.9 -179.8 -179.7 ... 179.8 179.9 180.0
Attributes:
    units:      km
    long_name:  distance to nearest neighbour

## Subsetting variables by time to match observations
We will use the `latitude` and `longitude` columns together with the `date` column from the crabeater seal observations to find the corresponding grid cell in the model outputs and extract the value of the environmental factor of our interest.

In [133]:
#Getting coordinates from the crabeater data
lat = xr.DataArray(crabeaters.latitude)
lon = xr.DataArray(crabeaters.longitude)
#Getting data of observation from the crabeater data
time = xr.DataArray(crabeaters.apply(lambda x: pd.to_datetime(f'{x.date}-16'), axis = 1))

## Extracting data

In [137]:
#Extracting data
var_sub = var_df.sel(time = time, yt_ocean = lat, xt_ocean = lon, method = 'nearest')

#Transforming to data frame
var_pd = var_sub.to_dataframe().sort_values(['time', 'xt_ocean', 'yt_ocean'])
#Adding year and month
var_pd['year'] = var_pd.time.dt.year
var_pd['month'] = var_pd.time.dt.month
#Removing time column that is no longer needed
var_pd.drop(columns = 'time', inplace = True)
#Finding name of columns to round up
round_cols = [i for i in var_pd.columns if 'ocean' in i]
#Rounding coordinate values prior to merging
var_pd = var_pd.round({round_cols[0]: 3, round_cols[1]: 3})
#Renaming variable to be added to crabeater data
var_pd.rename(columns = {varDict['var_mod']: varDict['var_short_name']}, inplace = True)
#Getting column names for merging
cols = var_pd.drop(columns = varDict['var_short_name']).columns.tolist()

#Checking results
print(cols); var_pd

['yt_ocean', 'xt_ocean', 'year', 'month']


,yt_ocean,xt_ocean,dist_ice_edge_km,year,month
dim_0,,,,,
1467,-65.058,108.95,141.925205,1978,1
1514,-65.015,108.95,145.578317,1978,1
1189,-65.353,109.65,95.851582,1978,1
1255,-65.269,109.65,103.435955,1978,1
1704,-64.846,109.65,144.633936,1978,1
...,...,...,...,...,...
1729,-64.803,-62.65,17.095099,2022,11
645,-66.916,-66.85,26.907984,2022,12
646,-66.916,-66.75,29.295693,2022,12


# Joining environmental data frame with crabeater data
We will use the grid cell coordinates and dates to perform this join.

In [138]:
crabeaters = crabeaters.merge(var_pd, on = cols, how = 'left')
crabeaters = crabeaters.drop_duplicates()
crabeaters

,event_date,date,latitude,longitude,source,year,sector,zone,yt_ocean,xt_ocean,...,SST_deg_C,bottom_temp_degC,SSS_psu,bottom_sal_psu,vel_lat_surf_msec,vel_lat_bottom_msec,vel_lon_surf_msec,vel_lon_bottom_msec,lt_pack_ice,dist_ice_edge_km
0,2004-02-19T00:00:00Z,2004-02,-77.697927,166.302939,GBIF_iNaturalist_Observations,2004,West Pacific,Antarctic,-77.687,166.35,...,-0.398438,-1.389679,33.180775,34.552128,0.083146,-0.003987,-0.054895,-0.011748,0.702381,775.634642
1,2014-01-13T14:51:22Z,2014-01,-76.859810,-36.055843,Filchner,2014,Atlantic,Antarctic,-76.842,-36.05,...,2.801361,-1.864105,34.046490,34.569469,-0.012738,-0.008299,0.022658,0.008238,0.690476,1280.861181
2,2000-01-06T00:00:00Z,2000-01,-76.775000,-157.290000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-157.25,...,-0.370117,1.146729,33.410400,34.672157,-0.040162,-0.001442,-0.026949,-0.001069,0.714286,445.928328
3,2000-01-06T00:00:00Z,2000-01,-76.766000,-156.850000,SCAR_Biology,2000,West Pacific,Antarctic,-76.758,-156.85,...,-0.404816,1.145599,33.329449,34.684376,-0.011817,-0.001766,-0.044433,-0.004446,0.714286,454.552151
4,2020-02-02T00:00:00Z,2020-02,-76.362262,163.916020,GBIF_NA_Observations,2020,West Pacific,Antarctic,-76.378,163.95,...,1.423065,-1.873322,34.118916,34.834370,0.193753,0.032634,-0.072459,-0.001650,0.785714,666.945871
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3237,1989-11-05T10:00:00Z,1989-11,-60.066700,70.200000,SCAR_seabirds,1989,Central Indian,Subantarctic,-60.047,70.25,...,-1.193298,-0.517700,33.778362,34.633255,-0.053467,-0.002976,0.110630,0.000508,0.000000,76.258827
3238,2008-12-26T00:00:00Z,2008-12,-59.640000,9.115000,GBIF_IRSNB_Belgian Antarctic Research Expediti...,2008,Atlantic,Antarctic,-59.645,9.15,...,-1.436432,-0.770233,33.698380,34.619843,0.046982,0.007774,0.076627,0.004839,0.321429,291.618185
3239,1989-11-05T02:00:00Z,1989-11,-59.483300,71.866700,SCAR_seabirds,1989,Central Indian,Subantarctic,-59.493,71.85,...,-1.240387,-0.523804,33.757450,34.632874,-0.031276,-0.000573,0.063193,-0.002624,0.000000,55.971258
3240,2013-11-12T00:00:00Z,2013-11,-57.826121,-26.346061,GBIF_NA_Observations,2013,Atlantic,Antarctic,-57.829,-26.35,...,-1.099792,0.045715,34.138268,34.659763,NaN,NaN,NaN,NaN,0.000000,129.334694


In [147]:
#Ensure output folder exists
os.makedirs(varDict['base_out'], exist_ok = True)

#Create file path where data will be saved
file_out = os.path.join(varDict['base_out'], 'unique_crabeater_obs_all_env.csv')
                        
crabeaters.to_csv(file_out, index = False)